# Haddendataset preprocessing

In [38]:
import os
import shutil
import sys

import numpy as np
from scipy import sparse
import pandas as pd
import math
import matplotlib.pyplot as plt
import random

### Load Data

In [39]:
### change `DATA_DIR` to the location where movielens-20m dataset sits
DATA_DIR = './'

In [40]:
cur_min_uc = 15
cur_val_num = 500
cur_te_num =500
cur_n_items_u = 15
cur_n_items_p = 50
dataset_index = 4
randomseed_num = 1
#cur_train_por = 7 
#cur_val_por = 3
#cur_test_por =3

In [41]:
raw_data_total = pd.read_csv(os.path.join(DATA_DIR, 'data.csv'), header=0)
print(raw_data_total.shape[0])
raw_data = pd.DataFrame({'count' : raw_data_total.groupby( [ "users", "products"] ).size()}).reset_index()
del raw_data['count']
print(raw_data.shape[0])
print(raw_data.shape)

539995
377922
(377922, 2)


In [42]:
a = raw_data_total.groupby("users").size()
print(a.shape)

(61192,)


In [43]:
b = raw_data_total.groupby("products").size()
print(b.shape)

(52570,)


In [33]:
raw_data.shape

(377922, 2)

### Data splitting procedure

get_count id 별로 그룹화, 그룹별 데이터 수 확인

In [34]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count

In [35]:
def filter_triplets(tp, min_uc=5, min_sc=0):
    # Only keep the triplets for items which were clicked on by at least min_sc users. 
    if min_sc > 0:
         #기존 data를 min_sc 해당되는 movield만 filtering       
        itemcount = get_count(tp, 'products')
        tp = tp[tp['products'].isin(itemcount.index[itemcount >= min_sc])]
    
    # Only keep the triplets for users who clicked on at least min_uc items
    # After doing this, some of the items will have less than min_uc users, but should only be a small proportion
    if min_uc > 0:
        usercount = get_count(tp, 'users')
        #기존 data를 min_uc이상 해당되는 userld만 filtering
        tp = tp[tp['users'].isin(usercount.index[usercount >= min_uc])]
    
    # Update both usercount and itemcount after filtering
    usercount, itemcount = get_count(tp, 'users'), get_count(tp, 'products') 
    return tp, usercount, itemcount

실제 data, 뽑아낸 usercont, 뽑아낸 movieId

In [36]:
raw_data, user_activity, item_popularity = filter_triplets(raw_data, 5, 0)

sparsity 구하기 raw_data의 열의 갯수(filtering 된 raw data 총 갯수) / userid 갯수 * products 갯수

In [37]:
sparsity = 1. * raw_data.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])

print("After filtering, there are %d watching events from %d users and %d products (sparsity: %.3f%%)" % 
      (raw_data.shape[0], user_activity.shape[0], item_popularity.shape[0], sparsity * 100))

After filtering, there are 300872 watching events from 12156 users and 47365 products (sparsity: 0.052%)


In [199]:
print(get_count(raw_data,'users').sort_values(ascending= False))

users
187     379
58      285
1459    262
394     258
181     244
       ... 
1153     15
6755     15
6719     15
6703     15
1433     15
Length: 2019, dtype: int64


In [200]:
def getportion(val, test):
    total = user_activity.index.shape[0]
    return round(((total-val-test)/total)*100), round((val/total)*100), round((test/total)*100)

In [201]:
def getDataindex(val,test):
    trainP,valP,testP = getportion(val,test)
    print(trainP, valP, testP)
    totalP = trainP + valP+ testP
    totalUser = user_activity.sort_values(ascending= False)
    sortedIndexUser = totalUser.reset_index()
    print(sortedIndexUser)
    #sortedIndexUser = sortedIndexUser[10:]
    print(sortedIndexUser)
    sortedIndex = sortedIndexUser['users']
    sortedIndex_tr = sortedIndex[(sortedIndex.index%totalP>=0) & (sortedIndex.index%totalP<trainP)]
    sortedIndex_va = sortedIndex[(sortedIndex.index%totalP>=trainP) & (sortedIndex.index%totalP<(trainP+valP))]
    sortedIndex_te = sortedIndex[(sortedIndex.index%totalP>=(trainP+valP)) & (sortedIndex.index%totalP<(trainP+valP+testP))]
    sorteduid = pd.concat([sortedIndex_tr,sortedIndex_va,sortedIndex_te])
    return sorteduid.tolist(), sortedIndex_tr.tolist(),sortedIndex_va.tolist(), sortedIndex_te.tolist()

In [202]:
def getDataindex_p(trainP,valP,testP):
    print(trainP, valP, testP)
    totalP = trainP + valP+ testP
    totalUser = user_activity.sort_values(ascending= False)
    sortedIndexUser = totalUser.reset_index()
    print(sortedIndexUser)
    #sortedIndexUser = sortedIndexUser[10:]
    print(sortedIndexUser)
    sortedIndex = sortedIndexUser['users']
    sortedIndex_tr = sortedIndex[(sortedIndex.index%totalP>=0) & (sortedIndex.index%totalP<trainP)]
    sortedIndex_va = sortedIndex[(sortedIndex.index%totalP>=trainP) & (sortedIndex.index%totalP<(trainP+valP))]
    sortedIndex_te = sortedIndex[(sortedIndex.index%totalP>=(trainP+valP)) & (sortedIndex.index%totalP<(trainP+valP+testP))]
    sorteduid = pd.concat([sortedIndex_tr,sortedIndex_va,sortedIndex_te])
    return sorteduid.tolist(), sortedIndex_tr.tolist(),sortedIndex_va.tolist(), sortedIndex_te.tolist()

In [203]:
def getDataindex_order_p(trainP,valP,testP):
    print(trainP, valP, testP)
    totalP = trainP + valP+ testP
    totalUser = user_activity.sort_values(ascending= False)
    sortedIndexUser = totalUser.reset_index()
    #sortedIndexUser = sortedIndexUser[10:]
    print(sortedIndexUser)
    sortedIndex = sortedIndexUser['users']
    tr_index = round(sortedIndexUser.shape[0]*(trainP/totalP))
    vr_index = round(tr_index +sortedIndexUser.shape[0]*(valP/totalP))
    print("trindex ", tr_index)
    print("vindex",vr_index)
    sortedIndex_tr = sortedIndex[:tr_index]
    sortedIndex_va = sortedIndex[tr_index:vr_index]
    sortedIndex_te = sortedIndex[vr_index:]
    sorteduid = pd.concat([sortedIndex_tr,sortedIndex_va,sortedIndex_te])
    return sorteduid.tolist(), sortedIndex_tr.tolist(),sortedIndex_va.tolist(), sortedIndex_te.tolist()

In [204]:
def RandomgetDataindex(val,test):
    trainP,valP,testP = getportion(val,test)
    print(trainP, valP, testP)
    totalP = trainP + valP+ testP
    totalUser = user_activity.sort_values(ascending= False)
    sortedIndexUser = totalUser.reset_index()
    #sortedIndexUser = sortedIndexUser[10:]
    sortedIndex = sortedIndexUser['users'].tolist()
    train = list()
    val = list()
    test = list()
    curarr = list()
    unique = list()
    for i in range(len(sortedIndex)):
        curarr.append(sortedIndex[i])
        if len(curarr) == totalP:
            curval = random.sample(curarr,valP)
            val += curval
            for k in curval :
                curarr.remove(k)
            testval = random.sample(curarr,testP)
            test += testval
            for k in testval :
                curarr.remove(k)
            train += curarr
            curarr.clear()
    if len(curarr) !=0 :
        train += curarr
    unique = train + val + test
    print(set(sortedIndex)-set(unique))
    print(set(train).intersection(val))
    print(set(train).intersection(test))
    print(set(val).intersection(test))
    return unique,train,val,test

unique_uid, tr_users, vd_users, te_users = getDataindex_order_p(cur_train_por,cur_val_por,cur_test_por)
print(unique_uid[0])
print(tr_users[0])
print(tr_users[-1])
print(vd_users[0])
print(vd_users[-1])
print(te_users[0])
print(te_users[-1])
print(unique_uid[-1])

unique_uid, tr_users, vd_users, te_users = getDataindex_p(cur_train_por,cur_val_por,cur_test_por)
print(len(unique_uid))
print(len(tr_users))
print(len(vd_users))
print(len(te_users))

In [205]:
unique_uid, tr_users, vd_users, te_users = RandomgetDataindex(cur_val_num,cur_te_num)
print(len(unique_uid))
print(len(tr_users))
print(len(te_users))
print(len(te_users))
print(tr_users[0])
print(vd_users[0])
print(te_users[0])

50 25 25
set()
set()
set()
set()
2019
1019
500
500
58
187
608


In [206]:
train_plays = raw_data.loc[raw_data['users'].isin(tr_users)]
print(pd.unique(train_plays['users']).shape)
print(train_plays.shape)

(1019,)
(46982, 2)


In [207]:
unique_sid = pd.unique(train_plays['products'])
print(unique_sid.shape)

(1156,)


In [208]:
print(len(unique_uid))
vad_plays = raw_data.loc[raw_data['users'].isin(vd_users)]
vad_before_set = set(pd.unique(vad_plays['users']))
vad_plays = vad_plays.loc[vad_plays['products'].isin(unique_sid)]
vad_set = set(pd.unique(vad_plays['users']))
remove_vad_uidx = vad_before_set - vad_set
print(remove_vad_uidx)
print(len(vad_before_set))
print(len(vad_set))
for x in remove_vad_uidx:
    if x in unique_uid:
                unique_uid.remove(x)
test_plays = raw_data.loc[raw_data['users'].isin(te_users)]
test_before_set = set(pd.unique(test_plays['users']))
test_plays = test_plays.loc[test_plays['products'].isin(unique_sid)]
test_set = set(pd.unique(test_plays['users']))
remove_test_uidx = test_before_set-test_set
print(len(unique_uid))
print(remove_test_uidx)
for x in remove_test_uidx:
    if x in unique_uid:
                unique_uid.remove(x)
print(len(test_before_set))
print(len(test_set))
print(len(unique_uid))
before_uid = len(unique_uid)
before_vd = pd.unique(vad_plays['users']).shape
before_te = pd.unique(test_plays['users']).shape

2019
set()
500
500
2019
set()
500
500
2019


In [209]:
def getSparsity(data):
    usercount = get_count(data,'users')
    itemcount = get_count(data, 'products')
    sparsity = 1. * data.shape[0] / (usercount.shape[0] * len(unique_sid))
    print("After filtering, there are %d watching events from %d users and %d products (sparsity: %.3f%%)" % 
      (data.shape[0], usercount.shape[0], len(unique_sid), sparsity * 100))   

In [210]:
getSparsity(train_plays)

After filtering, there are 46982 watching events from 1019 users and 1156 products (sparsity: 3.988%)


In [211]:
getSparsity(vad_plays)

After filtering, there are 23159 watching events from 500 users and 1156 products (sparsity: 4.007%)


In [212]:
getSparsity(test_plays)

After filtering, there are 22963 watching events from 500 users and 1156 products (sparsity: 3.973%)


In [213]:
def split_train_test_proportion(data,cur_num_items,test_prop=0.2):
    data_grouped_by_user = data.groupby('users')
    tr_list, te_list = list(), list()
    np.random.seed(12)

    for i, (index, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)
        if n_items_u >= cur_num_items:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else :
            if index in unique_uid:
                unique_uid.remove(index)
        if i % 1000 == 0:
            print("%d users sampled" % i)
            sys.stdout.flush()
    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)
    return data_tr, data_te

In [214]:
vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays,5)

0 users sampled


In [215]:
vad_palys_list = vad_plays_tr['users']

In [216]:
vad_palys_list_te = vad_plays_te['users']

In [217]:
print(len(tr_users))

1019


In [218]:
print(pd.unique(vad_plays_tr['users']).shape)
print(pd.unique(vad_plays_te['users']).shape)

(500,)
(500,)


In [219]:
test_plays_tr, test_plays_te = split_train_test_proportion(test_plays,5)

0 users sampled


In [220]:
print(pd.unique(test_plays_tr['users']).shape)

(500,)


In [221]:
print(pd.unique(test_plays_te['users']).shape)

(500,)


In [222]:
print(len(unique_uid))

2019


In [223]:
print(before_uid - len(unique_uid))
print(before_te[0] - pd.unique(test_plays_tr['users']).shape[0])
print(before_vd[0]- pd.unique(vad_plays_tr['users']).shape[0])

0
0
0


    unique_sid는 train_plays에서 뽑는 unique한 users

    unique_uid는 뒤섞인것 기준
    show2id는 index와 value를 바꾼 dict (unique_sid에서 뽑은 것 mapping) -> products : index
    profile2id는 index와 value를 바꾼 dict (전체 unique_uid에서 뽑은 것 mapping) -> users : index

In [224]:
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

    unique_sid는 unique_sid.txt로 기록 

In [225]:
pro_dir ='./module12'
if not os.path.exists(pro_dir):
    os.makedirs(pro_dir)

with open(os.path.join(pro_dir, 'unique_sid_'+str(dataset_index)+'.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

    split_train_test_proportion함수 data를 userId로 그룹화
    tr_list, te_list은 userId로 그룹화 된 data를 test_prop 만큼 비율로 나누어서 저장 
    te_list는 그룹화된 (int) data 갯수 * test_prop 개의 random한 index를 가지는 data 저장
    tr_list는 나머지 그룹화된 data 저장

### Save the data into (user_index, item_index) format

    numerize 함수 userId와 movieId를 index로 mapping 후 uid(users) sid(products)로 dataFrame생성

In [226]:
def numerize(tp):
    uid = list(map(lambda x: profile2id[x], tp['users']))
    sid = list(map(lambda x: show2id[x], tp['products']))
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [227]:
train_data = numerize(train_plays)
train_data.to_csv(os.path.join(pro_dir, 'train_'+str(dataset_index)+'.csv'), index=False)

In [228]:
def getSparsity_S(data):
    usercount = get_count(data,'uid')
    itemcount = get_count(data, 'sid')
    sparsity = 1. * data.shape[0] / (usercount.shape[0] * len(unique_sid))
    print("After filtering, there are %d watching events from %d users and %d products (sparsity: %.3f%%)" % 
      (data.shape[0], usercount.shape[0], len(unique_sid), sparsity * 100))   

In [229]:
getSparsity_S(train_data)

After filtering, there are 46982 watching events from 1019 users and 1156 products (sparsity: 3.988%)


In [230]:
vad_data_tr = numerize(vad_plays_tr)
vad_data_tr.to_csv(os.path.join(pro_dir, 'validation_tr_'+str(dataset_index)+'.csv'), index=False)

In [231]:
getSparsity_S(vad_data_tr)

After filtering, there are 18717 watching events from 500 users and 1156 products (sparsity: 3.238%)


In [232]:
vad_data_te = numerize(vad_plays_te)
vad_data_te.to_csv(os.path.join(pro_dir, 'validation_te_'+str(dataset_index)+'.csv'), index=False)

In [233]:
getSparsity_S(vad_data_te)

After filtering, there are 4442 watching events from 500 users and 1156 products (sparsity: 0.769%)


In [234]:
test_data_tr = numerize(test_plays_tr)
test_data_tr.to_csv(os.path.join(pro_dir, 'test_tr_'+str(dataset_index)+'.csv'), index=False)

In [235]:
getSparsity_S(test_data_tr)

After filtering, there are 18562 watching events from 500 users and 1156 products (sparsity: 3.211%)


In [236]:
test_data_te = numerize(test_plays_te)
test_data_te.to_csv(os.path.join(pro_dir, 'test_te_'+str(dataset_index)+'.csv'), index=False)

In [237]:
getSparsity_S(test_data_te)

After filtering, there are 4401 watching events from 500 users and 1156 products (sparsity: 0.761%)
